Import libraries

In [20]:
import pandas as pd

Source Kaggle Dataset: [US Accidents 2016-2023](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents)

In [21]:
accidents_df = pd.read_csv("D:/Programing/Weather2Go2/US_Accidents_March23.csv/US_Accidents_March23.csv")
accidents_df.shape


(7728394, 46)

In [22]:
accidents_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


Source Weather API: [Open Meteo](https://open-meteo.com/en/docs/historical-forecast-api?bounding_box=41.7,-90.5,48.3,-82.4&start_date=2016-01-01&end_date=2023-03-31&hourly=temperature_2m,weather_code,relative_humidity_2m,surface_pressure,visibility,wind_speed_10m,precipitation&temperature_unit=fahrenheit&wind_speed_unit=mph&precipitation_unit=inch&location_mode=csv_coordinates&csv_coordinates=42.3314,-83.0458%0A42.2808,-83.7430%0A42.7325,-84.5555%0A42.9634,-85.6681%0A43.0125,-83.6875%0A42.2917,-85.5872%0A44.7631,-85.6206%0A46.5436,-87.3954%0A46.4953,-84.3453%0A)

In [23]:
weather_df = pd.read_csv("D:/Programing/Weather2Go2/Data/mi_all_hourly_2016_2023.csv")
weather_df.shape

(1757232, 10)

In [24]:
weather_df.head()

,time,temperature_2m,weather_code,relative_humidity_2m,surface_pressure,visibility,wind_speed_10m,precipitation,latitude,longitude
0,2016-01-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3206,-83.0311
1,2016-01-01 06:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3206,-83.0311
2,2016-01-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3206,-83.0311
3,2016-01-01 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3206,-83.0311
4,2016-01-01 09:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.3206,-83.0311


In [25]:
weather_df = weather_df.dropna()
weather_df.shape

(1231032, 10)

Change Kaggle Weather Conditions to more minimal categories. 

In [26]:
CATS = [
  "clear","cloudy","fog","rain_light","rain_heavy","snow_light","snow_heavy",
  "freezing_rain","sleet_mix","thunder","hail","smoke_dust","severe","other","unknown"
]

def kaggle_weather_to_cat(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return "unknown"

    s = str(x).strip().lower()

    # remove common noise tokens but keep meaning
    # (we ignore "windy" as its own category; wind is handled by wind_speed feature)
    s = s.replace("/ windy", "").replace("windy", "").strip()

    # --- highest priority: truly severe phenomena ---
    if "tornado" in s or "funnel cloud" in s:
        return "severe"
    if "squall" in s:
        return "severe"
    if "duststorm" in s:
        return "severe"

    # --- hail (including small/light/heavy) ---
    if "hail" in s:
        return "hail"

    # --- thunder / t-storm ---
    if ("thunder" in s) or ("t-storm" in s) or ("tstorm" in s) or ("thunderstorm" in s):
        return "thunder"

    # --- freezing precip (most dangerous) ---
    # freezing drizzle/rain/fog -> treat as freezing risk bucket
    if "freezing" in s:
        return "freezing_rain"

    # --- sleet / ice pellets / wintry mix ---
    if ("sleet" in s) or ("ice pellet" in s) or ("wintry mix" in s) or ("rain and sleet" in s):
        return "sleet_mix"

    # --- snow (heavy vs light) ---
    if "snow" in s or "blowing snow" in s or "drifting snow" in s or "snow grains" in s:
        if "heavy" in s:
            return "snow_heavy"
        return "snow_light"

    # --- rain/drizzle/showers (heavy vs light) ---
    if ("rain" in s) or ("drizzle" in s) or ("shower" in s):
        if "heavy" in s:
            return "rain_heavy"
        return "rain_light"

    # --- fog/mist/haze ---
    if ("fog" in s) or ("mist" in s) or ("haze" in s):
        return "fog"

    # --- smoke/dust/sand/ash ---
    if ("smoke" in s) or ("dust" in s) or ("sand" in s) or ("ash" in s) or ("volcanic" in s):
        return "smoke_dust"

    # --- clouds ---
    if ("overcast" in s) or ("cloudy" in s) or ("scattered clouds" in s) or ("partly cloudy" in s) or ("mostly cloudy" in s):
        return "cloudy"

    # --- clear/fair ---
    if ("clear" in s) or (s == "fair") or s.startswith("fair "):
        return "clear"

    # --- explicit "n/a precipitation" should not force rain ---
    if "n/a precipitation" in s:
        return "other"

    return "other"


Change [NOAA Weather Code](https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM) to match simpler weather conditions.

In [27]:
def wmo_to_category(code: int) -> str:
    if code in [0]:
        return "clear"

    if code in [1, 2, 3]:
        return "cloudy"

    if code in [45, 48]:
        return "fog"

    # drizzle
    if code in [51, 53, 55]:
        return "rain_light"
    if code in [56, 57]:
        return "freezing_rain"

    # rain
    if code in [61, 63]:
        return "rain_light"
    if code in [65]:
        return "rain_heavy"
    if code in [66, 67]:
        return "freezing_rain"

    # snow
    if code in [71, 73, 77]:
        return "snow_light"
    if code in [75]:
        return "snow_heavy"

    # showers
    if code in [80, 81]:
        return "rain_light"
    if code in [82]:
        return "rain_heavy"
    if code in [85]:
        return "snow_light"
    if code in [86]:
        return "snow_heavy"

    # thunderstorms
    if code in [95]:
        return "thunder"
    if code in [96, 99]:
        return "hail"

    return "other"


In [ ]:
# map WMO codes -> categories on the standardized weather dataframe
weather_df['Weather_Category'] = weather_df['weather_code'].apply(wmo_to_category)

# map free-text conditions -> categories on the standardized accidents dataframe
accidents_df['Weather_Category'] = accidents_df['Weather_Condition'].apply(kaggle_weather_to_cat)

Subset to only Michigan accidents

In [29]:
accidents_mi = accidents_df[accidents_df['State'] == 'MI']

In [44]:
accidents_subset = accidents_mi[['Weather_Timestamp','Weather_Category','Temperature(F)','Weather_Condition','Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)','Precipitation(in)','Start_Lat', 'Start_Lng']].copy()


In [45]:
accidents_subset['Risk'] = 1
weather_df['Risk'] = 0

In [46]:
weather_rename = {
    "time": "Weather_Timestamp",
    "temperature_2m": "Temperature_F",
    "weather_code": "Weather_Code",
    "relative_humidity_2m": "Humidity_Pct",
    "surface_pressure": "Pressure_in",
    "visibility": "Visibility_mi",
    "wind_speed_10m": "Wind_Speed_mph",
    "precipitation": "Precipitation_in",
    "latitude": "Start_Lat",
    "longitude": "Start_Lng",
}

weather_std = weather_df.rename(columns=weather_rename)


In [47]:
accident_rename = {
    "Weather_Timestamp": "Weather_Timestamp",
    "Temperature(F)": "Temperature_F",
    "Weather_Condition": "Weather_Condition",
    "Humidity(%)": "Humidity_Pct",
    "Pressure(in)": "Pressure_in",
    "Visibility(mi)": "Visibility_mi",
    "Wind_Speed(mph)": "Wind_Speed_mph",
    "Precipitation(in)": "Precipitation_in",
    "Start_Lat": "Start_Lat",
    "Start_Lng": "Start_Lng",
}

accidents_std = accidents_subset.rename(columns=accident_rename)


In [48]:
weather_std.head()


,Weather_Timestamp,Temperature_F,Weather_Code,Humidity_Pct,Pressure_in,Visibility_mi,Wind_Speed_mph,Precipitation_in,Start_Lat,Start_Lng,Weather_Category,Risk
151940,2018-01-01 01:00:00+00:00,8.936600,0.0,59.0,1010.33563,60367.453,1.703648,0.0,42.3206,-83.0311,clear,0
151941,2018-01-01 02:00:00+00:00,7.316601,0.0,63.0,1010.25240,53149.605,2.110382,0.0,42.3206,-83.0311,clear,0
151942,2018-01-01 03:00:00+00:00,5.156601,0.0,67.0,1010.04285,45931.758,1.140651,0.0,42.3206,-83.0311,clear,0
151943,2018-01-01 04:00:00+00:00,2.636601,0.0,73.0,1009.91095,38713.910,1.414803,0.0,42.3206,-83.0311,clear,0
152684,2018-01-01 01:00:00+00:00,2.769800,1.0,58.0,999.05260,63320.210,3.607054,0.0,42.2923,-83.7368,cloudy,0


In [49]:
weather_copies = [weather_std.copy() for i in range(3)]

In [50]:
merged_df = pd.concat([accidents_std] + weather_copies, ignore_index=True, sort=False)

In [51]:
print(weather_std.shape)
print(accidents_subset.shape)
print(accidents_mi.shape)
print(merged_df.shape)

(1231032, 12)
(162191, 12)
(162191, 47)
(3855287, 13)


In [53]:
merged_df = merged_df.drop(columns=['Weather_Condition', 'Weather_Code'])
merged_df.head()

,Weather_Timestamp,Weather_Category,Temperature_F,Humidity_Pct,Pressure_in,Visibility_mi,Wind_Speed_mph,Precipitation_in,Start_Lat,Start_Lng,Risk
0,2016-11-30 15:53:00,cloudy,43.0,60.0,29.56,10.0,15.0,NaN,42.963215,-85.666740,1
1,2016-11-30 15:53:00,cloudy,43.0,60.0,29.56,10.0,15.0,NaN,42.955772,-85.659164,1
2,2016-11-30 16:07:00,cloudy,48.0,80.0,29.52,10.0,4.6,NaN,43.060432,-83.783752,1
3,2016-11-30 16:07:00,cloudy,48.0,80.0,29.52,10.0,4.6,NaN,42.942875,-83.648453,1
4,2016-11-30 15:53:00,cloudy,55.0,69.0,29.51,10.0,10.4,NaN,42.368423,-83.058128,1
